# Training Loop

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch 
import torch.optim as optim 
import torch.nn as nn 
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('fivethirtyeight')

In [3]:
%run -i data_generation/simple_linear_regression.py 
%run -i data_preparation/v0.py 
%run -i model_configuration/v0.py 

In [4]:
n_epochs = 1000

for epoch in range(n_epochs):
    model.train()

    yhat = model(x_train_tensor)

    loss = loss_fn(yhat, y_train_tensor)

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


## Higher Oreder Function

In [6]:
def exponentiation_builder(exponent):
    def skeleton_exponentiation(x):
        return x ** exponent
    return skeleton_exponentiation

myfunction = exponentiation_builder(2)
myfunction
myfunction(5)

25

In [7]:
def make_train_step_fn(model, loss_fn, optimizer):
    def perform_train_step_fn(x,y):
        model.train()
        # step1 computes predictions
        yhat = model(x)
        # step2 computes loss
        loss = loss_fn(yhat, y)
        # step3 computes gradients
        loss.backward()
        # step4 updates parameters
        optimizer.step()
        optimizer.zero_grad()

        return loss.item()
    return perform_train_step_fn

In [ ]:
%run -i data_preparation/v0.py 

In [10]:
%%writefile model_configuration/v1.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.1
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1,1)).to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean')

train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

Writing model_configuration/v1.py


In [11]:
%run -i model_configuration/v1.py

In [12]:
train_step_fn

<function __main__.make_train_step_fn.<locals>.perform_train_step_fn(x, y)>

In [13]:
%%writefile model_training/v1.py

n_epochs = 1000
losses = []
for epoch in range(n_epochs):
    loss = train_step_fn(x_train_tensor, y_train_tensor)
    losses.append(loss)

Writing model_training/v1.py


In [14]:
%run -i model_training/v1.py

In [15]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


## Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor 
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    def __len__(self):
        return len(self.x)
    
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

In [16]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713], device='cuda:0'), tensor([2.4745], device='cuda:0'))


## DataLoader

In [17]:
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
next(iter(train_loader))

[tensor([[0.2809],
         [0.3253],
         [0.1560],
         [0.5924],
         [0.0651],
         [0.8872],
         [0.4938],
         [0.0055],
         [0.1409],
         [0.0885],
         [0.1849],
         [0.7290],
         [0.8662],
         [0.3117],
         [0.6842],
         [0.1987]], device='cuda:0'),
 tensor([[1.5846],
         [1.8057],
         [1.2901],
         [2.1687],
         [1.1559],
         [2.8708],
         [1.9060],
         [1.0632],
         [1.1211],
         [1.0708],
         [1.5888],
         [2.4927],
         [2.6805],
         [1.7637],
         [2.3492],
         [1.2654]], device='cuda:0')]